# Whale Pivotality in Aave top Snapshot proposals

In [1]:
# sets up the pynb environment
from datetime import datetime
from zoneinfo import ZoneInfo
import os
import sys

from IPython.display import HTML
import pandas as pd

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from stages.data_processing.statistics import (
    get_number_of_whales_to_all_voters_ratio,
    get_score_comparisons,
)

In [2]:
print(f"Last updated on {datetime.now(ZoneInfo('UTC')).date()}")

Last updated on 2023-04-21


This article is part of a series of reports that aims to determine the impact of large token holders (whales) on DAO governance.

We take a look at the top 10 Aave Snapshot proposals ranked by voting power and voter count, including the first Butter Aave Elections.

We compare their voter turnout and pivotality metric (a measure of whale influence, described in our previous [Mirror article](https://mirror.xyz/butterd.eth/_VrHsBXZU0IYuIUMq0ifJTffvH0e1AKNh6I-BIV3Nc4) and [data report](https://butterymoney.github.io/gov_analysis/report/plutocracy_report.html)).

In [3]:
all_organization_proposals = pd.read_csv(
    "../plutocracy_data/full_report/Aave_report.csv.gzip",
    engine="c",
    low_memory=False,
    compression='gzip',
)
all_organization_proposals_filtered = pd.read_csv(
    "../plutocracy_data/full_report/Aave_report_filtered.csv.gzip",
    engine="c",
    low_memory=False,
    compression='gzip',
)


In [4]:
def to_organization_map(flat_organization_dataframe: pd.DataFrame):
    return {
        organization_name: proposal_df
        for organization_name, proposal_df in [
            (str(space_name), space_proposals)
            for space_name, space_proposals in flat_organization_dataframe.groupby(
                "proposal_space_name"
            )
        ]
    }

plutocracy_report_data = to_organization_map(all_organization_proposals)
plutocracy_report_data_filtered = to_organization_map(all_organization_proposals_filtered)


In [5]:
pd.set_option("display.max_rows", int(1e3))
score_differences = get_score_comparisons(
    plutocracy_report_data, plutocracy_report_data_filtered
)

In [6]:
score_differences_dfs: dict[str, pd.DataFrame] = dict()

initial_series_data = {
    organization: 0
    for organization in plutocracy_report_data.keys()
}
changed_outcome_proportions = pd.Series(initial_series_data, name="changed outcomes %")

for score_difference in score_differences:
    for organization, data in score_difference.items():
        data: dict[str, list] = data
        items = data.items()
        score_differences_dfs[organization] = pd.DataFrame(
            [score_data for _, score_data in items],
            index=pd.Index(
                ([id for id, _ in items]), name="Proposal"
            ),
            columns=[
                "proposal_id",
                "Title",
                "Date Started",
                "Date Ended",
                "score_differences",
                "whale_vp_proportion",
                "total_vp",
                "outcome_changed",
                "Result",
                "Result w/o Whales"
            ],
        ).astype({"total_vp": "float64"}, copy=False
        ).sort_values(["total_vp"], ascending=False)

        changed_outcome_proportions[organization] = score_differences_dfs[organization]["outcome_changed"].value_counts(normalize=True)[True]

        score_differences_dfs[organization].drop("outcome_changed", axis=1, inplace=True)

        score_differences_dfs[organization]["Date Started"] = pd.to_datetime(score_differences_dfs[organization]["Date Started"], utc=True, unit="s").dt.strftime("%d/%m/%y")
        score_differences_dfs[organization]["Date Ended"] = pd.to_datetime(score_differences_dfs[organization]["Date Ended"], utc=True, unit="s").dt.strftime("%d/%m/%y")

        space_id = plutocracy_report_data[organization].iloc[0]["proposal_space_id"]

        score_differences_dfs[organization]["total_vp"] = score_differences_dfs[
            organization
        ]["total_vp"].apply("{:.9f}".format)

        score_differences_dfs[organization]["Title"] = score_differences_dfs[organization].apply(
            lambda row: f'<a href=http://snapshot.org/#/{space_id}/proposal/{row.name} rel="noopener noreferrer" target="_blank">{row["Title"]}</a>',
            axis=1
        )
        score_differences_dfs[organization].style.format({"whale_vp_proportion": "{:.2%}".format})
changed_outcome_proportions = changed_outcome_proportions.apply(
    lambda proportion: "{:.2%}".format(proportion)
)

In [7]:
voting_ratios = get_number_of_whales_to_all_voters_ratio(
    plutocracy_report_data, plutocracy_report_data_filtered
)


## Synthesis

Show the percentage of proposals whose outcome changes if whales did not vote (i.e. *whale pivotality*).

In [8]:
dao_overview = pd.DataFrame(
    [list(result.items())[0][1] for result in voting_ratios],
    columns=[
        "# of whales",
        "all voters",
    ],
)
dao_overview.set_index(
    pd.Index([list(result.items())[0][0] for result in voting_ratios], name="DAO"),
    inplace=True
)

dao_overview.insert(2, changed_outcome_proportions.name, changed_outcome_proportions)
dao_overview

,# of whales,all voters,changed outcomes %
DAO,,,
Aave,1438,42586,9.33%


## Comparison of the Top 10 Proposals

In [33]:
top_10_list = [
    "0xf6da417983653f7e7f5560a8e16ad77d5e25533bba9157abfd222347b40d09d0",
    "0xde90f4d29c4c5dd55cba023b29bdde8c49e37f70c3398668a78e2f173a61b4e6",
    "0x84deca82139320b2570f04211b249e37b8a7602b4a0ed70e6fa772c9f6e94550",
    "0xc17bc863a09c72152c1ac25d09fcee4748b58eb2e10e70886e20e77ad4d8fad0",
    "0x743ae6001bb0fdfa9a8c4475a2489c20917a79084725f0101ac7c1e7f0cf27c6",
    "0x7e6175c21342dbd88dc5b0dabf2d0a86339e3ef3f199fc7d778a9a6f05ed0df9",
    "0xf4aec3fbab5096752be96f0e5b522f37318c1902cf8b897b049b7a94d478de73",
    "0x46cf72da892eb216edc1b7fe2f24f7491d8c37344b2b1f67632fa6950be034f7",
    "0x4579b562a4ea07c52871f724a8baf156514a97384e9427156d04fb6f0ae91515",
    "0xd8a400fc731b6e807f4d89aa1fa47225f0dfaaa8fc49ef04b46b11b98fbfa15f",
]  # credit @wwinter 🧈

mask = score_differences_dfs["Aave"].index.to_series().apply(
    lambda row: row in top_10_list
)

score_differences_dfs["Aave"] = score_differences_dfs["Aave"].loc[mask]
proposal_ids = score_differences_dfs["Aave"]["proposal_id"]
proposal_ids = pd.Categorical(proposal_ids, top_10_list)

score_differences_dfs["Aave"] = score_differences_dfs["Aave"].loc[proposal_ids]
score_differences_dfs["Aave"]["proposal_id"] = proposal_ids
score_differences_dfs["Aave"] = score_differences_dfs["Aave"].loc[proposal_ids].sort_values("proposal_id")

HTML(
    score_differences_dfs["Aave"]
    .rename(lambda index: score_differences_dfs["Aave"].loc[index]["Title"])
    .drop(["score_differences", "total_vp", "whale_vp_proportion", "Title", "proposal_id"], axis=1)
    .to_html(render_links=True, escape=False)
)


,Date Started,Date Ended,Result,Result w/o Whales
Proposal,,,,
[TEMP CHECK] Incentivized Delegate Campaign (3-month),03/04/23,10/04/23,TokenLogic,Blockworks Research
[TEMP CHECK] - Launch Aave V3 on Starknet,03/04/23,10/04/23,YEA,YEA
"[ARC] E-Mode Parameter Changes for V3 Avalanche, Optimism, Polygon, and Arbitrum",05/04/23,12/04/23,Option 3,Option 3
[TEMP CHECK] - Whitelist Stargate for V3 Portals,13/04/23,18/04/23,YAE,YAE
Safety Module upgrade to v1.5,15/03/23,20/03/23,YES,YES
Deploy Aave V3 to Metis Andromeda Mainnet,17/03/23,23/03/23,YAE,YAE
[ARFC] Add MKR to Ethereum v3,08/03/23,13/03/23,YAE,YAE
[TEMP CHECK] Aave V3 Deployment on zkSync Era Mainnet,12/04/23,15/04/23,YAE,YAE
[TEMP CHECK] Aave V3 deployment on BNB Chain,29/03/23,01/04/23,YAE,YAE


<sup>Source for proposal list: https://dune.com/butterd/aave-top-10-snapshot-proposals</sup>

### Proportion of Outcomes Changed

In [10]:
print(f"{changed_outcome_proportions['Aave']} of Aave's proposal outcomes (of the 150 sampled) change after filtering out whale voting power.")

9.33% of Aave's proposal outcomes (of the 150 sampled) change after filtering out whale voting power.


---

Case Studies
---

#### [TEMP CHECK] Incentivized Delegate Campaign (3-month)

Aka the first Butter'd Aave Election.

**Proposal ID**: [0xf6da417](https://snapshot.org/#/aave.eth/proposal/0xf6da417983653f7e7f5560a8e16ad77d5e25533bba9157abfd222347b40d09d0)

**Start Date**: 03/04/23
**End Date**: 10/04/23

**Voting System**: Weighted Voting

**Voting Strategies**:
- aAAVE
- AAVE + stkAAVE
- Matic AAVE + amAAVE
- AAVE in stkBPT (Balancer Pool Token)



Aave votes to select a delegate who will receive a $15k grant (paid in AAVE) from Aave Grants after a three-month term as a delegate, a delegate election. TokenLogic ultimately won this election, however, if we remove whale voting power from this proposal we find that Blockworks Research would've been selected by the Aave community instead.

In [18]:
propsal_choices = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0xf6da417983653f7e7f5560a8e16ad77d5e25533bba9157abfd222347b40d09d0'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Aave"].index.to_series().apply(
    lambda s: "0xf6da417" in s,
)
proposal_score_differences = score_differences_dfs["Aave"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0xf6da417983653f7e7f5560a8e16ad77d5e25533bba9157abfd222347b40d09d0'].iloc[0]['proposal_scores']


result = pd.DataFrame(
    {choice: [score, score_diff, score - score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Whale-only Scores", "Non-whale Scores"],
)

result

,Blockworks Research,ConsenSys,Curia,DAOStewards,Fire Eyes,Flipside Crypto,FranklinDAO,OnChainCoop,Oxytocin,Saludiego201.eth,StableLab,TokenLogic,Wallfacer Labs
Scores,87232.635594,17620.565593,12742.462405,434.508563,104834.493934,90345.343934,66127.639535,409.610873,30504.177435,437.382397,146246.026838,185171.775039,435.859890
Whale-only Scores,86487.989188,16950.926291,12712.705752,407.073261,104802.547664,90318.258696,65886.207100,391.862324,30491.294636,393.737487,146194.584615,185153.921653,420.744217
Non-whale Scores,744.646406,669.639302,29.756653,27.435302,31.946270,27.085238,241.432435,17.748550,12.882799,43.644910,51.442223,17.853386,15.115673


#### [TEMP CHECK] - Launch Aave V3 on Starknet


**Proposal ID**: [0xde90f4d](https://snapshot.org/#/aave.eth/proposal/0xde90f4d29c4c5dd55cba023b29bdde8c49e37f70c3398668a78e2f173a61b4e6)

**Start Date**: 03/04/23
**End Date**: 10/04/23

**Voting System**: Single Choice

**Voting Strategies**:
- aAAVE
- AAVE + stkAAVE
- Matic AAVE + amAAVE
- AAVE in stkBPT (Balancer Pool Token)



This is a proposal to gauge the Aave community’s sentiments on going on with phase 2 of the Aave’s integration on Starknet. Its outcome did not change after filtering out whales.

In [20]:
propsal_choices = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0xde90f4d29c4c5dd55cba023b29bdde8c49e37f70c3398668a78e2f173a61b4e6'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Aave"].index.to_series().apply(
    lambda s: "0xde90f4d" in s,
)
proposal_score_differences = score_differences_dfs["Aave"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0xde90f4d29c4c5dd55cba023b29bdde8c49e37f70c3398668a78e2f173a61b4e6'].iloc[0]['proposal_scores']


result = pd.DataFrame(
    {choice: [score, score_diff, score - score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Whale-only Scores", "Non-whale Scores"],
)

result

,YEA,NOP,ABSTAIN
Scores,664848.446683,21.627573,136.417016
Whale-only Scores,407757.376934,4.772256,120.558800
Non-whale Scores,257091.069749,16.855317,15.858216


#### [ARC] E-Mode Parameter Changes for V3 Avalanche, Optimism, Polygon, and Arbitrum


**Proposal ID**: [0x84deca8](https://snapshot.org/#/aave.eth/proposal/0x84deca82139320b2570f04211b249e37b8a7602b4a0ed70e6fa772c9f6e94550)

**Start Date**: 05/04/23
**End Date**: 11/04/23

**Voting System**: Single Choice

**Voting Strategies**:
- aAAVE
- AAVE + stkAAVE
- Matic AAVE + amAAVE
- AAVE in stkBPT (Balancer Pool Token)



This proposal aims to optimize stablecoin [E-Mode](https://docs.aave.com/developers/whats-new/efficiency-mode-emode) parameters on various non-Ethereum chains. Its outcome did not change after filtering out whales.

In [21]:
propsal_choices = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x84deca82139320b2570f04211b249e37b8a7602b4a0ed70e6fa772c9f6e94550'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Aave"].index.to_series().apply(
    lambda s: "0x84deca8" in s,
)
proposal_score_differences = score_differences_dfs["Aave"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x84deca82139320b2570f04211b249e37b8a7602b4a0ed70e6fa772c9f6e94550'].iloc[0]['proposal_scores']


result = pd.DataFrame(
    {choice: [score, score_diff, score - score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Whale-only Scores", "Non-whale Scores"],
)

result

,Option 1,Option 2,Option 3,Option 4,Abstain,Nay
Scores,126545.660433,166.034309,532246.554661,140.712281,489.850798,2.067035
Whale-only Scores,84143.973524,83.499083,291145.238949,107.559375,416.456718,1.017788
Non-whale Scores,42401.686910,82.535226,241101.315713,33.152905,73.394080,1.049248


#### [TEMP CHECK] - Whitelist Stargate for V3 Portals


**Proposal ID**: [0xc17bc86](https://snapshot.org/#/aave.eth/proposal/0xc17bc863a09c72152c1ac25d09fcee4748b58eb2e10e70886e20e77ad4d8fad0)

**Start Date**: 13/04/23
**End Date**: 18/04/23

**Voting System**: Single Choice

**Voting Strategies**:
- aAAVE
- AAVE + stkAAVE
- Matic AAVE + amAAVE
- AAVE in stkBPT (Balancer Pool Token)



Proposes that Aave whitelist Stargate on their [Portal](https://docs.aave.com/developers/whats-new/portal) bridge. Its outcome did not change after filtering out whales.

In [23]:
propsal_choices = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0xc17bc863a09c72152c1ac25d09fcee4748b58eb2e10e70886e20e77ad4d8fad0'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Aave"].index.to_series().apply(
    lambda s: "0xc17bc86" in s,
)
proposal_score_differences = score_differences_dfs["Aave"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0xc17bc863a09c72152c1ac25d09fcee4748b58eb2e10e70886e20e77ad4d8fad0'].iloc[0]['proposal_scores']


result = pd.DataFrame(
    {choice: [score, score_diff, score - score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Whale-only Scores", "Non-whale Scores"],
)

result

,YAE,NAY,ABSTAIN
Scores,654682.761288,16.887568,167.351060
Whale-only Scores,364423.857267,8.134670,143.935683
Non-whale Scores,290258.904021,8.752898,23.415377


#### Safety Module upgrade to v1.5

**Proposal ID**: [0x743ae60](https://snapshot.org/#/aave.eth/proposal/0x743ae6001bb0fdfa9a8c4475a2489c20917a79084725f0101ac7c1e7f0cf27c6)

**Start Date**: 15/03/23
**End Date**: 20/03/23

**Voting System**: Single Choice Voting


**Voting Strategies**:
- aAAVE
- AAVE + stkAAVE
- Matic AAVE + amAAVE
- AAVE in stkBPT (Balancer Pool Token)

A proposal to gauge the community's sentiments on upgrading the Safety Module smart contracts (stkAAVE, stkABPT) to v1.5. The outcome of this proposal did not change after filtering out whales.

In [ ]:
propsal_choices = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x743ae6001bb0fdfa9a8c4475a2489c20917a79084725f0101ac7c1e7f0cf27c6'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Aave"].index.to_series().apply(
    lambda s: "0x743ae60" in s,
)
proposal_score_differences = score_differences_dfs["Aave"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x743ae6001bb0fdfa9a8c4475a2489c20917a79084725f0101ac7c1e7f0cf27c6'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff, score - score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Whale-only Scores", "Non-whale Scores"],
)

#### Deploy Aave V3 to Metis Andromeda Mainnet

**Proposal ID**: [0x7e6175c](https://snapshot.org/#/aave.eth/proposal/0x7e6175c21342dbd88dc5b0dabf2d0a86339e3ef3f199fc7d778a9a6f05ed0df9)

**Start Date**: 17/03/23
**End Date**: 23/03/23

**Voting System**: Single Choice Voting


**Voting Strategies**:
- aAAVE
- AAVE + stkAAVE
- Matic AAVE + amAAVE
- AAVE in stkBPT (Balancer Pool Token)



This is a proposal to gauge the community's sentiments to deploy Aave V3 to the Metis "Andromeda" Mainnet. The outcome of this proposal did not change after filtering out whales.

In [29]:
propsal_choices = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x7e6175c21342dbd88dc5b0dabf2d0a86339e3ef3f199fc7d778a9a6f05ed0df9'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Aave"].index.to_series().apply(
    lambda s: "0x7e6175c" in s,
)
proposal_score_differences = score_differences_dfs["Aave"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x7e6175c21342dbd88dc5b0dabf2d0a86339e3ef3f199fc7d778a9a6f05ed0df9'].iloc[0]['proposal_scores']
pd.DataFrame(
    {choice: [score, score_diff, score - score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Whale-only Scores", "Non-whale Scores"],
)

,YAE,NAY,ABSTAIN
Scores,555698.763981,162135.341400,140.725517
Whale-only Scores,133249.462649,161953.210564,137.650972
Non-whale Scores,422449.301332,182.130836,3.074545


#### [ARFC] Add MKR to Ethereum v3

**Proposal ID**: [0xf4aec3f](https://snapshot.org/#/aave.eth/proposal/0xf4aec3fbab5096752be96f0e5b522f37318c1902cf8b897b049b7a94d478de73)

**Start Date**: 08/03/23
**End Date**: 13/03/23

**Voting System**: Single Choice Voting


**Voting Strategies**:
- aAAVE
- AAVE + stkAAVE
- Matic AAVE + amAAVE
- AAVE in stkBPT (Balancer Pool Token)



A proposal to onboard MKR to the Aave V3 Liquidity Pool. The outcome of this proposal did not change after filtering out whales.

In [31]:
propsal_choices = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0xf4aec3fbab5096752be96f0e5b522f37318c1902cf8b897b049b7a94d478de73'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Aave"].index.to_series().apply(
    lambda s: "0xf4aec3f" in s,
)
proposal_score_differences = score_differences_dfs["Aave"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0xf4aec3fbab5096752be96f0e5b522f37318c1902cf8b897b049b7a94d478de73'].iloc[0]['proposal_scores']
pd.DataFrame(
    {choice: [score, score_diff, score - score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Whale-only Scores", "Non-whale Scores"],
)

,YAE,NAE,ABSTAIN
Scores,660079.717273,12.211159,19.760356
Whale-only Scores,421665.918408,8.137339,17.155329
Non-whale Scores,238413.798865,4.073820,2.605028


#### [TEMP CHECK] Aave V3 Deployment on zkSync Era Mainnet


**Proposal ID**: [0x46cf72d](https://snapshot.org/#/aave.eth/proposal/0x46cf72da892eb216edc1b7fe2f24f7491d8c37344b2b1f67632fa6950be034f7)

**Start Date**: 12/04/23
**End Date**: 15/04/23

**Voting System**: Single Choice

**Voting Strategies**:
- aAAVE
- AAVE + stkAAVE
- Matic AAVE + amAAVE
- AAVE in stkBPT (Balancer Pool Token)



A temperature check to gauge the community’s sentiments on deploying Aave V3 on <a class="reference external" href="https://zksync.io/">zkSync Era</a> Mainnet. This proposal’s outcome also was not changed after filtering out whales.

In [26]:
propsal_choices = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x46cf72da892eb216edc1b7fe2f24f7491d8c37344b2b1f67632fa6950be034f7'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Aave"].index.to_series().apply(
    lambda s: "0x46cf72d" in s,
)
proposal_score_differences = score_differences_dfs["Aave"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x46cf72da892eb216edc1b7fe2f24f7491d8c37344b2b1f67632fa6950be034f7'].iloc[0]['proposal_scores']


result = pd.DataFrame(
    {choice: [score, score_diff, score - score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Whale-only Scores", "Non-whale Scores"],
)

result

,YAE,NAY,ABSTAIN
Scores,577937.331903,1133.242375,134.422288
Whale-only Scores,529876.707255,1.350356,1.585889
Non-whale Scores,48060.624648,1131.892020,132.836399


#### [TEMP CHECK] Aave V3 deployment on BNB Chain


**Proposal ID**: [0x4579b56](https://snapshot.org/#/aave.eth/proposal/0x4579b562a4ea07c52871f724a8baf156514a97384e9427156d04fb6f0ae91515)

**Start Date**: 29/03/23
**End Date**: 01/04/23

**Voting System**: Single Choice

**Voting Strategies**:
- aAAVE
- AAVE + stkAAVE
- Matic AAVE + amAAVE
- AAVE in stkBPT (Balancer Pool Token)



A temperature check to gauge the community's sentiments on deploying Aave V3 on BNB Chain. Its outcome did not change after filtering out whales.

In [27]:
propsal_choices = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x4579b562a4ea07c52871f724a8baf156514a97384e9427156d04fb6f0ae91515'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Aave"].index.to_series().apply(
    lambda s: "0x4579b56" in s,
)
proposal_score_differences = score_differences_dfs["Aave"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0x4579b562a4ea07c52871f724a8baf156514a97384e9427156d04fb6f0ae91515'].iloc[0]['proposal_scores']


result = pd.DataFrame(
    {choice: [score, score_diff, score - score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Whale-only Scores", "Non-whale Scores"],
)

result

,YAE,NAY,ABSTAIN
Scores,583635.461921,2296.049067,124.059645
Whale-only Scores,191783.269653,1039.979339,113.040863
Non-whale Scores,391852.192268,1256.069728,11.018781


#### [ARFC] Grant OP `Emission_Admin` for Optimism v3 Liquidity Pool to Lido DAO


**Proposal ID**: [0xd8a400f](https://snapshot.org/#/aave.eth/proposal/0xd8a400fc731b6e807f4d89aa1fa47225f0dfaaa8fc49ef04b46b11b98fbfa15f)

**Start Date**: 08/03/23
**End Date**: 13/03/23

**Voting System**: Single Choice

**Voting Strategies**:
- aAAVE
- AAVE + stkAAVE
- Matic AAVE + amAAVE
- AAVE in stkBPT (Balancer Pool Token)



This proposal aims to enable Lido DAO to distribute OP rewards to the Aave V3 Optimism Liquitdity Pool. Its outcome did not change after filtering out whales.

In [32]:
propsal_choices = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0xd8a400fc731b6e807f4d89aa1fa47225f0dfaaa8fc49ef04b46b11b98fbfa15f'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Aave"].index.to_series().apply(
    lambda s: "0xd8a400f" in s,
)
proposal_score_differences = score_differences_dfs["Aave"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Aave'][plutocracy_report_data['Aave']['proposal_id'] == '0xd8a400fc731b6e807f4d89aa1fa47225f0dfaaa8fc49ef04b46b11b98fbfa15f'].iloc[0]['proposal_scores']


result = pd.DataFrame(
    {choice: [score, score_diff, score - score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Whale-only Scores", "Non-whale Scores"],
)

result

,YAE,NAE,ABSTAIN
Scores,578961.637041,6.087976,133.073240
Whale-only Scores,248331.876540,1.009792,122.666259
Non-whale Scores,330629.760501,5.078184,10.406981
